In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import pre_processing as pp
from importlib import reload
K.set_image_dim_ordering('th')

/home/nima2021996/anaconda3/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training = pp.get_training()

In [3]:
test = pp.get_test()

In [7]:
X_train = np.array([x for x, y in training])
y_train = np.array([y for x, y in training])
X_test = test

In [8]:
pixel_count = X_train.shape[1] * X_train.shape[2]

In [9]:
# Normalize values (0-1)
X_train = np.divide(X_train, 255)
X_test = np.divide(X_test, 255)
y_train = np_utils.to_categorical(y_train)

In [10]:
class_count = y_train.shape[1]

In [11]:
# Seed for reproducibility
seed = 42
np.random.seed(seed)

In [12]:
# Flatten
# X_train = X_train.reshape(X_train.shape[0], pixel_count).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], pixel_count).astype('float32')

In [13]:
# BASE NEURAL NETWORK MODEL
# def base_nn():
#     nn_model = Sequential()
#     # Relu hidden layer, same number of neurons as input layer
#     nn_model.add(Dense(pixel_count, input_dim=pixel_count, kernel_initializer='normal', activation='relu'))
#     # Softmax output layer
#     nn_model.add(Dense(class_count, kernel_initializer='normal', activation='softmax'))
#     # Compile
#     nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return nn_model

In [14]:
# # Create a base neural network model
# model = base_nn()
# model.fit(X_train, y_train_vec, validation_data=(X_test, y_test_vec), epochs=10, batch_size=50, verbose=2)
# scores = model.evaluate(X_test, y_test_vec, verbose=0)
# print("Baseline Error %.2f%%" % (100-scores[1]*100))

In [15]:
# FOR CNN
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [16]:
def base_cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
cnn_model = base_cnn()
cnn_model.fit(X_train, y_train, epochs=100, batch_size=200, verbose=2)

Epoch 1/100
 - 131s - loss: 0.8086 - acc: 0.7740
Epoch 2/100
 - 130s - loss: 0.5454 - acc: 0.8609
Epoch 3/100
 - 130s - loss: 0.4841 - acc: 0.8762
Epoch 4/100
 - 130s - loss: 0.4471 - acc: 0.8833
Epoch 5/100
 - 131s - loss: 0.4157 - acc: 0.8900
Epoch 6/100
 - 130s - loss: 0.3877 - acc: 0.8960
Epoch 7/100
 - 129s - loss: 0.3632 - acc: 0.9017
Epoch 8/100
 - 128s - loss: 0.3371 - acc: 0.9066
Epoch 9/100
 - 129s - loss: 0.3123 - acc: 0.9117
Epoch 10/100
 - 129s - loss: 0.2927 - acc: 0.9146
Epoch 11/100
 - 128s - loss: 0.2682 - acc: 0.9207
Epoch 12/100
 - 129s - loss: 0.2477 - acc: 0.9250
Epoch 13/100
 - 129s - loss: 0.2253 - acc: 0.9312
Epoch 14/100
 - 129s - loss: 0.2084 - acc: 0.9352
Epoch 15/100
 - 127s - loss: 0.1864 - acc: 0.9418
Epoch 16/100
 - 128s - loss: 0.1689 - acc: 0.9459
Epoch 17/100
 - 129s - loss: 0.1550 - acc: 0.9490
Epoch 18/100
 - 128s - loss: 0.1348 - acc: 0.9555
Epoch 19/100
 - 129s - loss: 0.1211 - acc: 0.9598
Epoch 20/100
 - 129s - loss: 0.1104 - acc: 0.9631
Epoch 21/

In [18]:
def larger_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
cnn_model_larger = larger_cnn()
cnn_model_larger.fit(X_train, y_train, epochs=100, batch_size=200, verbose=2)

Epoch 1/100
 - 149s - loss: 0.9858 - acc: 0.7096
Epoch 2/100
 - 150s - loss: 0.6174 - acc: 0.8385
Epoch 3/100
 - 149s - loss: 0.5521 - acc: 0.8588
Epoch 4/100
 - 149s - loss: 0.5132 - acc: 0.8688
Epoch 5/100
 - 149s - loss: 0.4910 - acc: 0.8761
Epoch 6/100
 - 149s - loss: 0.4714 - acc: 0.8800
Epoch 7/100
 - 150s - loss: 0.4563 - acc: 0.8838
Epoch 8/100
 - 150s - loss: 0.4438 - acc: 0.8880
Epoch 9/100
 - 149s - loss: 0.4315 - acc: 0.8891
Epoch 10/100
 - 149s - loss: 0.4216 - acc: 0.8901
Epoch 11/100
 - 150s - loss: 0.4140 - acc: 0.8932
Epoch 12/100
 - 148s - loss: 0.4061 - acc: 0.8945
Epoch 13/100
 - 149s - loss: 0.3957 - acc: 0.8955
Epoch 14/100
 - 149s - loss: 0.3861 - acc: 0.8981
Epoch 15/100
 - 149s - loss: 0.3800 - acc: 0.8981
Epoch 16/100
 - 148s - loss: 0.3754 - acc: 0.8992
Epoch 17/100
 - 148s - loss: 0.3672 - acc: 0.9009
Epoch 18/100
 - 149s - loss: 0.3618 - acc: 0.9013
Epoch 19/100
 - 149s - loss: 0.3560 - acc: 0.9034
Epoch 20/100
 - 148s - loss: 0.3458 - acc: 0.9055
Epoch 21/

In [21]:
cnn_pred = cnn_model.predict(X_test)

In [24]:
cnn_pred_nums = np.array([np.argmax(pred) for pred in cnn_pred])

In [28]:
cnn_pred_final = np.array([(i, x) for (i, x) in enumerate(cnn_pred_nums)])

In [31]:
np.savetxt('../data/test_y.csv', cnn_pred_final, delimiter=',', fmt='%d', header='Id,Label')

In [32]:
cnn_large_pred = cnn_model_larger.predict(X_test)

In [34]:
cnn_large_pred_nums = np.array([np.argmax(pred) for pred in cnn_large_pred])

In [35]:
cnn_large_pred_final = np.array([(i, x) for (i, x) in enumerate(cnn_large_pred_nums)])

In [37]:
np.savetxt('../data/test_y_largecnn.csv', cnn_large_pred_final, delimiter=',', fmt='%d', header='Id,Label')